In [22]:
import urllib.request  # импортируем модуль 
import re
import urllib.parse
import os
import time
import requests
from requests.exceptions import HTTPError
#from mystem import analyze

In [23]:
def take_Meta(html, url): #извлекает мета-информацию
    meta = {}
    dating = ''
    pub = ''
    regPostTitle = re.compile('h2 class="itemTitle">(.*?)</h2>', flags= re.DOTALL)
    date = re.compile('<span class="itemDateCreated">.*, (\d* [А-Яа-я]* 20.*) \d.*', flags= re.DOTALL)
    authorship = re.compile('<a rel="author" href=".*">(.*?)</a>.*<div class="itemBody">', flags= re.DOTALL)
    item = re.compile('class="itemCategory">.*<span>Опубликовано в</span>.*<a href="/news/[A-Za-z]*">(.*?)</a>', flags= re.DOTALL)
    author = authorship.search(html)
    titles = regPostTitle.search(html)
    title = titles[1].replace('\n\t\t\t\n\t  \t', '').replace('\n\t  \t\n\t  ', '')
    fin_date = date.search(html)
    category = item.search(html)
    #переводим время создания в адекватный вид
    time_pub = fin_date[1].split(' ')
    if time_pub[1] != '':
        if time_pub[1] == "Январь":
            pub = "01"
        if time_pub[1] == "Февраль":
            pub = "02"
        if time_pub[1] == "Март":
            pub = "03"
        if time_pub[1] == "Апрель":
            pub = "04"
        if time_pub[1] == "Май":
            pub = "05"
        if time_pub[1] == "Июнь":
            pub = "06"
        if time_pub[1] == "Июль":
            pub = "07"
        if time_pub[1] == "Август":
            pub = "08"
        if time_pub[1] == "Сентябрь":
            pub = "09"
        if time_pub[1] == "Октябрь":
            pub = "10"
        if time_pub[1] == "Ноябрь":
            pub = "11"
        if time_pub[1] == "Декабрь":
            pub = "11"
    dating = str(time_pub[0]) + "." + str(pub) + '.' + str(time_pub[2])
    meta[title] = [author[1], dating, category[1], url, time_pub[2]]
    print(meta)
    return meta

In [24]:
def creating_Meta(meta): #функция создает словарь и потом заносит данные в цсв
   # titles_for_table = []
   # raw = ''
    for key, value in meta.items():
        if value[0] == []:
            value[0] == None
        print("Название статьи: %s\nАвтор: %s\nДата: %s\nКатегория: %s\nURL: %s\nГод публикации: %s" 
              % (key, value[0], value[1], value[2], value[3], value[4]))
    #    raw = '%s\t%s\t%s\t%s\tпублицистика\t\t%s\tнейтральный\tн-возраст\tн-уровень\tрайонная\t%s\tназвание газеты\t%s\tгазета\tРоссия\tкакой-то регион\tru'
    #with open('metadata.csv', 'a', encoding='utf-8') as file:
    #    file.write(row % ('тут ссылка', 'Петя', 'Название статьи', '26.09.2016', 'образование', 'url', '2016'))
    return

In [25]:
def article_Text(html): #функция чистит текст статьи
    #print(html)
    finding_text = re.compile('<div class="itemBody">.*<!-- Item introtext -->(.*)<!-- Item Rating -->', flags= re.DOTALL)
    text = finding_text.search(html)
    final_t = str(text[1])
    regTag = re.compile('<.*?>', re.DOTALL)  # это рег. выражение находит все тэги
    regScript = re.compile('<script>.*?</script>', re.DOTALL) # все скрипты
    regComment = re.compile('<!--.*?-->', re.DOTALL)  # все комментарии
    specsymbols_1 = re.compile('&[A-Za-z]*;', re.DOTALL)
    # а дальше заменяем ненужные куски на пустую строку
    clean_t = regScript.sub("", final_t)
    clean_t = regComment.sub("", clean_t)
    clean_t = regTag.sub("", clean_t)
    clean_text = specsymbols_1.sub("", clean_t)
   # print(clean_text)
    return clean_text

In [26]:
def textnTags(clean_text, meta, number): #создает файл с текстом
    text = str(clean_text)
    tags = ''
    for key, value in meta.items():
        ti = key
        au = value[0]
        da = value[1]
        topic = value[2]
        url = value[3]
        year = value[4]
    dates = da.split('.')
    mounth = dates[1]
    filename = "article_" + str(number) + '.txt'
    tags = "@au " + au + "\n@ti "+ ti + "\n@da " + da + "\n@topic " + topic + "\n@url " + url
    dirname = "plain"
  #  dirname_2 = "mystem-xml"
  #  dirname_3 = "mystem-plain"
    path_dirname = "%s" % dirname
   # path_dirn2 = "%s" % dirname_2
   # path_dirn3 = "%s" % dirname_3
    if not os.path.exists(path_dirname):
        os.makedirs(path_dirname)
  #  if not os.path.exists(path_dirn2):
  #      os.makedirs(path_dirn2)
  #  if not os.path.exists(path_dirn3):
  #      os.makedirs(path_dirn3)
    dirname_year = "%s" % year
    path_year = os.path.join(path_dirname, dirname_year)
   # path_year2 = os.path.join(path_dirn2, dirname_year)
   # path_year3 = os.path.join(path_dirn3, dirname_year)
    if not os.path.exists(path_year):
        os.makedirs(path_year)
  #  if not os.path.exists(path_year2):
  #      os.makedirs(path_year2)
  #  if not os.path.exists(path_year3):
  #      os.makedirs(path_year3)
    dirname_mounth = "%s" % mounth
    path_mounth = os.path.join(path_year, dirname_mounth)
   # path_moun2 = os.path.join(path_year2, dirname_mounth)
   # path_moun3 = os.path.join(path_year3, dirname_mounth)
    if not os.path.exists(path_mounth):
        os.makedirs(path_mounth)
   # if not os.path.exists(path_moun2):
   #     os.makedirs(path_moun2)
    if not os.path.exists(path_moun3):
        os.makedirs(path_moun3)
    filepath = os.path.join(path_mounth, filename).encode('ascii')
    #print(filepath, type(filepath))
   # filepath2 = os.path.join(path_moun2, filename).encode('ascii')
   # filepath3 = os.path.join(path_moun3, filename).encode('ascii')
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(tags)
  #  with open(filepath2, 'w') as f:
  #      f.write(tags)
  #  with open(filepath3, 'w') as f:
  #      f.write(tags)
    with open(filepath, 'a') as file:
        file.write(text)
    return filepath

In [27]:
def creating_Csv(filepath, meta):
    row = ''
    path = str(filepath)
    for key, value in meta.items():
        author = value[0]
        header = key
        created = value[1]
        topic = value[2]
        source = value[3]
        year = value[4]
    row = '%s\t%s\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\tн-уровень\tрайонная\t%s\tВолжская правда\t%s\tгазета\tРоссия\tВолжский\tru\n'
    with open('metadata.csv', 'a+', encoding='utf-8') as file:
        file.write(row % (path, author, header, created, topic, source, year))
    return 

In [28]:
def mystem(meta, number):
    for key, value in meta.items():
        da = value[1]
        year = value[4]
    dates = da.split('.')
    mounth = dates[1]
    filename = "article_" + str(number) + '.txt'
    dirname_2 = "mystem-xml"
    dirname_3 = "mystem-plain"
    path_dirn2 = "%s" % dirname_2
    path_dirn3 = "%s" % dirname_3
    if not os.path.exists(path_dirn2):
        os.makedirs(path_dirn2)
    if not os.path.exists(path_dirn3):
        os.makedirs(path_dirn3)
    dirname_year = "%s" % year
    path_year2 = os.path.join(path_dirn2, dirname_year)
    path_year3 = os.path.join(path_dirn3, dirname_year)
    if not os.path.exists(path_year2):
        os.makedirs(path_year2)
    if not os.path.exists(path_year3):
        os.makedirs(path_year3)
    dirname_mounth = "%s" % mounth
    path_moun2 = os.path.join(path_year2, dirname_mounth)
    path_moun3 = os.path.join(path_year3, dirname_mounth)
    if not os.path.exists(path_moun2):
        os.makedirs(path_moun2)
    if not os.path.exists(path_moun3):
        os.makedirs(path_moun3)
    filepath2 = os.path.join(path_moun2, filename).encode('ascii')
    filepath3 = os.path.join(path_moun3, filename).encode('ascii')
    with open(filepath2, 'w') as f:
        f.write(tags)
    with open(filepath3, 'w') as f:
        f.write(tags)
    return

In [29]:
def main(): #запускающая функция
    row = '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'
    with open('metadata.csv', 'w', encoding='utf-8') as file: 
        file.write(row % ("path", "author", "header",
                          "created", "sphere",
                          "topic", "style", "audience_age", "audience_level",
                          "audience_size", "source", "publication", "publ_year", "medium", 
                          "country", "region", "language"))
    commonUrl = 'http://gazeta-vp.ru/news/./item/'
    for i in range(17943, 17950):
        pageUrl = commonUrl + str(i)
        try:
            r = requests.get(pageUrl)
            r.raise_for_status()
        except HTTPError:
            print (r.url, 'Could not download page')
        else:
            print (r.url, 'downloaded successfully')
            url =  r.url
            number = i
            user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером
            req = urllib.request.Request(url, headers={'User-Agent':user_agent})  
            with urllib.request.urlopen(req) as response:
                html = response.read().decode('utf-8')
            take = take_Meta(html, url)
            meta = creating_Meta(take)
            text = article_Text(html)
            text_with_tg = textnTags(text, take, number)
            csv = creating_Csv(text_with_tg, take)
            stem = mystem(take, number)
main()